In [2]:
# generate_reports.py

import os
import io
from pathlib import Path
from datetime import datetime

# Data and DB Libraries
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

# Reporting and Plotting Libraries
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# --- CONFIGURATION ---
# The folder where all reports will be saved
OUTPUT_DIR = Path("GeneratedReports")
# The prefix for learners we are interested in
LEARNER_PREFIX = "BT"

# --- HELPER FUNCTIONS ---

def add_df_to_doc(doc, df, title=""):
    """Adds a Pandas DataFrame to a docx document with a title."""
    if title:
        doc.add_heading(title, level=2)
    
    # Add a table to the document
    if df.empty:
        doc.add_paragraph("No data available.")
        return
        
    # Reset index if it's not meaningful (like 0, 1, 2, ...)
    if isinstance(df.index, pd.RangeIndex):
        df_for_table = df.reset_index(drop=True)
    else:
        df_for_table = df.reset_index()

    table = doc.add_table(rows=1, cols=len(df_for_table.columns))
    table.style = 'Table Grid'
    
    # Add the header rows.
    for j, col_name in enumerate(df_for_table.columns):
        table.cell(0, j).text = str(col_name)

    # Add the rest of the data frame
    for i, row in df_for_table.iterrows():
        row_cells = table.add_row().cells
        for j, val in enumerate(row):
            row_cells[j].text = str(val)

def create_plot_from_df(df, title, y_label, x_col='Decision Point'):
    """Creates a line plot from a dataframe and returns it as an in-memory image."""
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 5))
    
    for col in df.columns:
        if col != x_col:
            ax.plot(df[x_col], df[col], marker='o', linestyle='-', label=col)
    
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(x_col, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(rotation=45, ha='right')
    fig.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout to make room for legend
    
    # Save plot to an in-memory buffer
    img_stream = io.BytesIO()
    plt.savefig(img_stream, format='png', dpi=300)
    plt.close(fig) # Close the plot to free memory
    img_stream.seek(0)
    return img_stream


# --- CORE SCRIPT LOGIC ---

def fetch_analytics_data(supabase: Client) -> pd.DataFrame:
    """Fetches and cleans data from the historical_learning_analytics table."""
    print(f"Fetching data for learners starting with '{LEARNER_PREFIX}'...")
    try:
        response = supabase.table('historical_learning_analytics').select(
            'created_at, user_id, goal_id, scenario_attempt_number, decision_number, kc_scores_after_decision, metric_values_after_decision, users!inner(name)'
        ).like(
            'users.name', f'{LEARNER_PREFIX}%'
        ).order(
            'user_id'
        ).order(
            'created_at', desc=False  # <-- THIS IS THE CORRECTED LINE
        ).execute()

        if not response.data:
            print("No data found for the specified learners.")
            return pd.DataFrame()

        df = pd.DataFrame(response.data)
        
        # Unpack user name from the nested dictionary
        df['user_name'] = df['users'].apply(lambda x: x['name'])
        df = df.drop(columns=['users'])
        
        print(f"Successfully fetched {len(df)} records.")
        return df

    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return pd.DataFrame()


def process_learner_data(raw_df: pd.DataFrame) -> dict:
    """Processes the raw DataFrame into a dictionary of DataFrames, one for each learner."""
    print("Processing raw data...")
    if raw_df.empty:
        return {}

    all_learners_data = {}
    
    # Get all unique KC and Metric names across the entire dataset
    all_kcs = set()
    all_metrics = set()
    raw_df['kc_scores_after_decision'].dropna().apply(lambda x: all_kcs.update(x.keys()))
    raw_df['metric_values_after_decision'].dropna().apply(lambda x: all_metrics.update(x.keys()))

    # Group data by each user
    for user_id, group in raw_df.groupby('user_id'):
        user_name = group['user_name'].iloc[0]
        print(f"  -> Processing data for {user_name} ({user_id})")

        # Sort chronologically for this user
        learner_df = group.sort_values('created_at').reset_index(drop=True)
        
        # Create a clean 'Decision Point' identifier for plotting
        learner_df['Decision Point'] = [f"G{r.goal_id}-S{r.scenario_attempt_number}-D{r.decision_number}" for r in learner_df.itertuples()]

        # Expand the JSON columns into actual dataframe columns
        kc_df = pd.json_normalize(learner_df['kc_scores_after_decision']).reindex(columns=sorted(list(all_kcs))).fillna(0)
        metric_df = pd.json_normalize(learner_df['metric_values_after_decision']).reindex(columns=sorted(list(all_metrics))).fillna(0)

        # Combine everything into a single, clean DataFrame for the learner
        processed_df = pd.concat([
            learner_df[['user_id', 'user_name', 'created_at', 'Decision Point']],
            kc_df,
            metric_df
        ], axis=1)

        all_learners_data[user_id] = {
            "user_name": user_name,
            "full_data": processed_df,
            "all_kcs": sorted(list(all_kcs)),
            "all_metrics": sorted(list(all_metrics))
        }
        
    print("Data processing complete.")
    return all_learners_data


def generate_individual_report(learner_data: dict, output_dir: Path):
    """Generates a single Word document report for one learner."""
    user_name = learner_data['user_name']
    df = learner_data['full_data']
    kcs = learner_data['all_kcs']
    metrics = learner_data['all_metrics']
    
    print(f"  -> Generating report for {user_name}...")
    
    doc = Document()
    doc.add_heading(f'Learner Report: {user_name}', 0)
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # --- Section 1: Final Scores ---
    doc.add_heading('Final Cumulative Scores', level=1)
    final_scores = df.iloc[-1]
    
    # Final KC Scores Table
    final_kc_scores = final_scores[kcs].to_frame(name='Final Score')
    final_kc_scores.index.name = "Knowledge Component (KC)"
    add_df_to_doc(doc, final_kc_scores, "Final KC Scores")

    # Final Metric Scores Table
    final_metric_scores = final_scores[metrics].to_frame(name='Final Value')
    final_metric_scores.index.name = "Metric"
    add_df_to_doc(doc, final_metric_scores, "Final Metric Scores")
    
    doc.add_page_break()

    # --- Section 2: Scores Over Time ---
    doc.add_heading('Performance Over Time', level=1)
    
    # KC Scores Progression
    kc_progression_df = df[['Decision Point'] + kcs]
    add_df_to_doc(doc, kc_progression_df.set_index('Decision Point'), "KC Scores per Decision Point")
    
    kc_plot_img = create_plot_from_df(kc_progression_df, "KC Scores Over Time", "KC Score")
    doc.add_picture(kc_plot_img, width=Inches(6.5))
    
    # Metric Scores Progression
    metric_progression_df = df[['Decision Point'] + metrics]
    add_df_to_doc(doc, metric_progression_df.set_index('Decision Point'), "Metric Values per Decision Point")
    
    metric_plot_img = create_plot_from_df(metric_progression_df, "Metric Values Over Time", "Metric Value")
    doc.add_picture(metric_plot_img, width=Inches(6.5))

    # --- Save the document ---
    file_path = output_dir / f"Report_{user_name.replace(' ', '_')}.docx"
    doc.save(file_path)


def generate_master_report(all_learners_data: dict, output_dir: Path):
    """Generates the master report with aggregate stats and leaderboards."""
    print("-> Generating Master Report...")
    if not all_learners_data:
        print("   No data available to generate a master report.")
        return

    doc = Document()
    doc.add_heading('Master Learner Report', 0)
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    doc.add_paragraph(f"This report covers {len(all_learners_data)} learners whose names start with '{LEARNER_PREFIX}'.")

    # --- Compile final scores from all learners into one DataFrame ---
    final_scores_list = []
    for user_id, data in all_learners_data.items():
        final_row = data['full_data'].iloc[-1].copy()
        final_scores_list.append(final_row)
    
    final_scores_df = pd.DataFrame(final_scores_list).set_index('user_name')
    kcs = all_learners_data[list(all_learners_data.keys())[0]]['all_kcs']
    metrics = all_learners_data[list(all_learners_data.keys())[0]]['all_metrics']

    # --- Section 1: Aggregate Statistics ---
    doc.add_heading('Aggregate Statistics (Final Scores)', level=1)
    
    # KC Stats
    kc_stats = final_scores_df[kcs].describe().loc[['mean', 'min', 'max']].round(2)
    add_df_to_doc(doc, kc_stats, "KC Score Statistics (Across All Learners)")

    # Metric Stats
    metric_stats = final_scores_df[metrics].describe().loc[['mean', 'min', 'max']].round(2)
    add_df_to_doc(doc, metric_stats, "Metric Value Statistics (Across All Learners)")
    
    doc.add_page_break()
    
    # --- Section 2: Leaderboards ---
    doc.add_heading('Leaderboards', level=1)
    
    # KC Leaderboard (based on sum of all final KC scores)
    final_scores_df['Total KC Score'] = final_scores_df[kcs].sum(axis=1)
    kc_leaderboard = final_scores_df[['Total KC Score']].sort_values('Total KC Score', ascending=False).round(2)
    add_df_to_doc(doc, kc_leaderboard.head(10), "Top 10 - KC Leaderboard (by Sum of Final KC Scores)")
    
    # Metric Leaderboard (based on a normalized composite score)
    # Normalizing is important because metrics have different scales (e.g., Revenue vs. Reputation)
    # We scale each metric to be between 0 and 1, then sum them up for a composite score.
    doc.add_paragraph(
        "The Metric Leaderboard is calculated by normalizing each metric (scaling from 0 to 1 based on the min/max in this group) "
        "and then summing these normalized scores. This gives a balanced view of overall performance across all metrics."
    )
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    normalized_metrics = scaler.fit_transform(final_scores_df[metrics])
    final_scores_df['Composite Metric Score'] = pd.DataFrame(normalized_metrics, index=final_scores_df.index).sum(axis=1)
    metric_leaderboard = final_scores_df[['Composite Metric Score']].sort_values('Composite Metric Score', ascending=False).round(3)
    add_df_to_doc(doc, metric_leaderboard.head(10), "Top 10 - Metric Leaderboard (by Composite Score)")

    # --- Save the document ---
    file_path = output_dir / "Master_Report.docx"
    doc.save(file_path)
    print("   Master Report saved.")


def main():
    """Main function to run the entire reporting script."""
    # --- Setup ---
    print("Starting report generation script...")
    load_dotenv('.env.local')
    
    supabase_url = os.environ.get("NEXT_PUBLIC_SUPABASE_URL")
    supabase_key = os.environ.get("NEXT_PUBLIC_SUPABASE_ANON_KEY")

    if not supabase_url or not supabase_key:
        print("Error: Supabase URL or Key not found in .env.local file.")
        print("Please create a .env.local file with your credentials.")
        return

    # Create output directory if it doesn't exist
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    # Initialize Supabase client
    supabase: Client = create_client(supabase_url, supabase_key)

    # --- Data Fetching and Processing ---
    raw_df = fetch_analytics_data(supabase)
    if raw_df.empty:
        print("Exiting script as no data was fetched.")
        return
        
    all_learners_data = process_learner_data(raw_df)
    
    if not all_learners_data:
        print("Exiting script as no learner data could be processed.")
        return

    # --- Report Generation ---
    print("\nGenerating individual learner reports...")
    for user_id in all_learners_data:
        generate_individual_report(all_learners_data[user_id], OUTPUT_DIR)
    
    print("\nGenerating master report...")
    generate_master_report(all_learners_data, OUTPUT_DIR)
    
    print(f"\n✅ All reports have been generated and saved in the '{OUTPUT_DIR}' folder.")


if __name__ == "__main__":
    main()

Starting report generation script...
Fetching data for learners starting with 'BT'...
Successfully fetched 107 records.
Processing raw data...
  -> Processing data for BT8BT9 (0dcaf179-58b2-411b-8874-d41c18467735)
  -> Processing data for BT15BT16 (1193bb01-6be2-4e97-b603-dda1b811446d)
  -> Processing data for BT14 (14b9c968-dc54-4752-a254-8351e461117a)
  -> Processing data for BT2 (3184fa5a-11bf-4526-b34b-cfaae969ab29)
  -> Processing data for BT4 (3541ed59-254e-41b8-b208-f4c9f7bdb95a)
  -> Processing data for BT13 (50730893-f080-4f52-94cb-8b4d438609b8)
  -> Processing data for BT7 (63eb2958-1758-4f6d-89f5-2158ac6873f5)
  -> Processing data for BT1 (65c037c6-9bdb-4e0d-9f18-ac5d3f7281c9)
  -> Processing data for BT11BT12 (701c22f8-fd72-4f51-b46e-d0cadb761db8)
  -> Processing data for BT7 (9b65c028-a111-4b49-9bae-1154f9c72326)
  -> Processing data for BT5 (bb07dc77-3422-472a-8de3-4cc4ef2fd088)
  -> Processing data for BT3+BT6 (d276ef1e-e84d-4b91-b05c-9a55e41d93f5)
  -> Processing data f

In [5]:
# generate_reports.py

import os
import io
from pathlib import Path
from datetime import datetime, timedelta

# Data and DB Libraries
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

# Reporting and Plotting Libraries
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
OUTPUT_DIR = Path("GeneratedReports")
LEARNER_PREFIX = "BT"

# --- HELPER FUNCTIONS ---

def add_df_to_doc(doc, df, title=""):
    """Adds a rounded Pandas DataFrame to a docx document with a title."""
    if title:
        doc.add_heading(title, level=2)
    
    if df.empty:
        doc.add_paragraph("No data available.")
        return
        
    # Round all numeric columns to 2 decimal places
    df_rounded = df.copy()
    for col in df_rounded.select_dtypes(include=['number']).columns:
        df_rounded[col] = df_rounded[col].round(2)

    # Reset index if it's a simple range, otherwise keep it as a column
    if isinstance(df_rounded.index, pd.RangeIndex):
        df_for_table = df_rounded.reset_index(drop=True)
    else:
        df_for_table = df_rounded.reset_index()

    table = doc.add_table(rows=1, cols=len(df_for_table.columns))
    table.style = 'Table Grid'
    
    for j, col_name in enumerate(df_for_table.columns):
        table.cell(0, j).text = str(col_name)

    for i, row in df_for_table.iterrows():
        row_cells = table.add_row().cells
        for j, val in enumerate(row):
            row_cells[j].text = str(val)

def create_plot_from_df(df, title, y_label, x_col='Decision Point'):
    """Creates a line plot from a dataframe and returns it as an in-memory image."""
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 5.5))
    
    # Plotting logic
    for col in df.columns:
        if col != x_col:
            ax.plot(df[x_col], df[col], marker='o', linestyle='-', label=col)
    
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(x_col, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)
    
    # Adjust legend and layout
    if len(df.columns) > 2: # More than one line to plot
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.tight_layout(rect=[0, 0, 0.85, 1])
    else:
        fig.tight_layout()
        
    plt.xticks(rotation=45, ha='right')
    
    img_stream = io.BytesIO()
    plt.savefig(img_stream, format='png', dpi=300)
    plt.close(fig)
    img_stream.seek(0)
    return img_stream

# --- CORE SCRIPT LOGIC ---

def fetch_analytics_data(supabase: Client) -> pd.DataFrame:
    """Fetches and cleans data, including the goal name."""
    print(f"Fetching data for learners starting with '{LEARNER_PREFIX}'...")
    try:
        # UPDATED QUERY: Joins with goals table to get goal name
        response = supabase.table('historical_learning_analytics').select(
            'created_at, user_id, goal_id, scenario_attempt_number, decision_number, '
            'kc_scores_after_decision, metric_values_after_decision, '
            'users!inner(name), goals!inner(name)'
        ).like(
            'users.name', f'{LEARNER_PREFIX}%'
        ).order(
            'user_id'
        ).order(
            'created_at', desc=False
        ).execute()

        if not response.data:
            print("No data found for the specified learners.")
            return pd.DataFrame()

        df = pd.DataFrame(response.data)
        
        # Unpack nested data
        df['user_name'] = df['users'].apply(lambda x: x['name'])
        df['goal_name'] = df['goals'].apply(lambda x: x['name'])
        df = df.drop(columns=['users', 'goals'])
        
        # Convert created_at to datetime objects for proper sorting
        df['created_at'] = pd.to_datetime(df['created_at'])
        
        print(f"Successfully fetched {len(df)} records.")
        return df

    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return pd.DataFrame()

def process_learner_data(raw_df: pd.DataFrame) -> dict:
    """Processes raw data, adds a 'Genesis' point, and structures it by learner."""
    print("Processing raw data...")
    if raw_df.empty:
        return {}

    all_learners_data = {}
    
    # Get all unique KC and Metric names across the entire dataset
    all_kcs = set()
    all_metrics = set()
    raw_df['kc_scores_after_decision'].dropna().apply(lambda x: all_kcs.update(x.keys()))
    raw_df['metric_values_after_decision'].dropna().apply(lambda x: all_metrics.update(x.keys()))
    all_kcs, all_metrics = sorted(list(all_kcs)), sorted(list(all_metrics))

    for (user_id, goal_id), group in raw_df.groupby(['user_id', 'goal_id']):
        user_name = group['user_name'].iloc[0]
        goal_name = group['goal_name'].iloc[0]
        
        print(f"  -> Processing data for {user_name} on Goal: '{goal_name}'")

        learner_df = group.sort_values('created_at').reset_index(drop=True)
        
        # Expand JSON columns
        kc_df = pd.json_normalize(learner_df['kc_scores_after_decision']).reindex(columns=all_kcs).fillna(0)
        metric_df = pd.json_normalize(learner_df['metric_values_after_decision']).reindex(columns=all_metrics).fillna(0)
        
        # Create a clean 'Decision Point' identifier
        learner_df['Decision Point'] = [f"G{r.goal_id}-S{r.scenario_attempt_number}-D{r.decision_number}" for r in learner_df.itertuples()]

        # --- ADD GENESIS POINT ---
        genesis_row = {
            'created_at': learner_df['created_at'].iloc[0] - timedelta(seconds=1),
            'Decision Point': 'Genesis',
            **{kc: 0 for kc in all_kcs},
            **{metric: 0 for metric in all_metrics}
        }
        genesis_df = pd.DataFrame([genesis_row])
        
        # Combine everything into a clean DataFrame for the learner
        base_info = learner_df[['created_at', 'Decision Point']]
        processed_df = pd.concat([base_info, kc_df, metric_df], axis=1)
        
        # Prepend the Genesis row
        full_df = pd.concat([genesis_df, processed_df], ignore_index=True)
        
        # Identify active KCs and Metrics (ones that changed from 0)
        final_scores = full_df.iloc[-1]
        active_kcs = [kc for kc in all_kcs if final_scores[kc] != 0]
        active_metrics = [metric for metric in all_metrics if final_scores[metric] != 0]

        all_learners_data[f"{user_id}_{goal_id}"] = {
            "user_name": user_name,
            "goal_name": goal_name,
            "full_data": full_df,
            "all_kcs": all_kcs,
            "all_metrics": all_metrics,
            "active_kcs": active_kcs,
            "active_metrics": active_metrics
        }
        
    print("Data processing complete.")
    return all_learners_data

def calculate_aggregate_stats(all_learners_data: dict) -> dict:
    """Calculates aggregate statistics (mean, min, max) across all learners."""
    print("Calculating aggregate statistics...")
    if not all_learners_data:
        return {}

    final_scores_list = []
    for data in all_learners_data.values():
        final_row = data['full_data'].iloc[-1].copy()
        final_row['user_name'] = data['user_name']
        final_scores_list.append(final_row)
    
    if not final_scores_list:
        return {}

    final_scores_df = pd.DataFrame(final_scores_list).set_index('user_name')
    kcs = all_learners_data[list(all_learners_data.keys())[0]]['all_kcs']
    metrics = all_learners_data[list(all_learners_data.keys())[0]]['all_metrics']

    kc_stats = final_scores_df[kcs].describe().loc[['mean', 'min', 'max']]
    metric_stats = final_scores_df[metrics].describe().loc[['mean', 'min', 'max']]

    return {"kc": kc_stats, "metric": metric_stats, "final_scores_df": final_scores_df}

def generate_individual_report(learner_data: dict, agg_stats: dict, output_dir: Path):
    """Generates a single, enhanced Word document report for one learner."""
    user_name = learner_data['user_name']
    goal_name = learner_data['goal_name']
    df = learner_data['full_data']
    active_kcs = learner_data['active_kcs']
    active_metrics = learner_data['active_metrics']
    
    print(f"  -> Generating report for {user_name} (Goal: {goal_name})...")
    
    doc = Document()
    doc.add_heading(f'Learner Report: {user_name}', 0)
    
    # --- THIS IS THE CORRECTED LINE ---
    # Use 'Quote' which is a built-in paragraph style, often italicized.
    doc.add_paragraph(f"Goal Attempted: {goal_name}", style='Quote') 
    # --- END OF CORRECTION ---

    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # --- Section 1: Final Scores with Comparison ---
    doc.add_heading('Final Cumulative Scores', level=1)
    doc.add_paragraph("This section shows the learner's final score for each item compared to the class average, minimum, and maximum.")
    
    final_scores = df.iloc[-1]

    # Enhanced Final KC Scores Table
    if active_kcs:
        kc_summary = final_scores[active_kcs].to_frame(name='Your Score')
        kc_summary = kc_summary.join(agg_stats['kc'][active_kcs].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
        kc_summary.index.name = "Knowledge Component (KC)"
        add_df_to_doc(doc, kc_summary, "Final KC Scores")

    # Enhanced Final Metric Scores Table
    if active_metrics:
        metric_summary = final_scores[active_metrics].to_frame(name='Your Value')
        metric_summary = metric_summary.join(agg_stats['metric'][active_metrics].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
        metric_summary.index.name = "Metric"
        add_df_to_doc(doc, metric_summary, "Final Metric Values")
    
    doc.add_page_break()

    # --- Section 2: Scores Over Time ---
    doc.add_heading('Performance Over Time', level=1)
    
    # KC Scores Progression
    if active_kcs:
        kc_progression_df = df[['Decision Point'] + active_kcs]
        add_df_to_doc(doc, kc_progression_df.set_index('Decision Point'), "KC Scores per Decision Point")
        kc_plot_img = create_plot_from_df(kc_progression_df, "KC Scores Over Time", "KC Score")
        doc.add_picture(kc_plot_img, width=Inches(6.5))
    
    # Metric Scores Progression (SPLIT PLOTS)
    if active_metrics:
        metric_progression_df = df[['Decision Point'] + active_metrics]
        add_df_to_doc(doc, metric_progression_df.set_index('Decision Point'), "Metric Values per Decision Point")

        # Revenue Plot
        if 'Revenue' in active_metrics:
            revenue_df = df[['Decision Point', 'Revenue']]
            revenue_plot_img = create_plot_from_df(revenue_df, "Revenue Over Time", "Revenue Value")
            doc.add_heading("Revenue Progression", level=3)
            doc.add_picture(revenue_plot_img, width=Inches(6.5))

        # Other Metrics Plot
        other_metrics = [m for m in active_metrics if m != 'Revenue']
        if other_metrics:
            other_metrics_df = df[['Decision Point'] + other_metrics]
            other_metrics_plot_img = create_plot_from_df(other_metrics_df, "Other Metric Values Over Time", "Metric Value")
            doc.add_heading("Other Metrics Progression", level=3)
            doc.add_picture(other_metrics_plot_img, width=Inches(6.5))

    file_path = output_dir / f"Report_{user_name.replace(' ', '_')}_{goal_name.replace(' ', '_')}.docx"
    doc.save(file_path)

def generate_master_report(agg_stats: dict, all_kcs: list, all_metrics: list, num_learners: int, output_dir: Path):
    """Generates the master report with aggregate stats and leaderboards."""
    print("-> Generating Master Report...")
    if not agg_stats:
        print("   No data available to generate a master report.")
        return

    doc = Document()
    doc.add_heading('Master Learner Report', 0)
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    doc.add_paragraph(f"This report covers {num_learners} learner-goal attempts for names starting with '{LEARNER_PREFIX}'.")

    # --- Section 1: Aggregate Statistics ---
    doc.add_heading('Aggregate Statistics (Final Scores)', level=1)
    add_df_to_doc(doc, agg_stats['kc'].T, "KC Score Statistics (Across All Learners)")
    add_df_to_doc(doc, agg_stats['metric'].T, "Metric Value Statistics (Across All Learners)")
    
    doc.add_page_break()
    
    # --- Section 2: Leaderboards ---
    doc.add_heading('Leaderboards', level=1)
    final_scores_df = agg_stats['final_scores_df']
    
    # KC Leaderboard
    final_scores_df['Total KC Score'] = final_scores_df[all_kcs].sum(axis=1)
    kc_leaderboard = final_scores_df[['Total KC Score']].sort_values('Total KC Score', ascending=False)
    add_df_to_doc(doc, kc_leaderboard.head(10), "Top 10 - KC Leaderboard (by Sum of Final KC Scores)")
    
    # Metric Leaderboard
    try:
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        normalized_metrics = scaler.fit_transform(final_scores_df[all_metrics])
        final_scores_df['Composite Metric Score'] = pd.DataFrame(normalized_metrics, index=final_scores_df.index).sum(axis=1)
        metric_leaderboard = final_scores_df[['Composite Metric Score']].sort_values('Composite Metric Score', ascending=False)
        
        doc.add_paragraph(
            "The Metric Leaderboard is calculated by normalizing each metric (scaling from 0 to 1 based on the min/max in this group) "
            "and then summing these normalized scores. This gives a balanced view of overall performance across all metrics."
        )
        add_df_to_doc(doc, metric_leaderboard.head(10), "Top 10 - Metric Leaderboard (by Composite Score)")
    except ImportError:
        doc.add_paragraph("Metric Leaderboard could not be generated. Please install scikit-learn: pip install scikit-learn")

    doc.save(output_dir / "Master_Report.docx")
    print("   Master Report saved.")

def main():
    """Main function to run the entire reporting script."""
    print("Starting report generation script...")
    load_dotenv('.env.local')
    
    supabase_url = os.environ.get("NEXT_PUBLIC_SUPABASE_URL")
    supabase_key = os.environ.get("NEXT_PUBLIC_SUPABASE_ANON_KEY")

    if not supabase_url or not supabase_key:
        print("Error: Supabase URL or Key not found. Please check your .env.local file.")
        return

    OUTPUT_DIR.mkdir(exist_ok=True)
    supabase: Client = create_client(supabase_url, supabase_key)

    raw_df = fetch_analytics_data(supabase)
    if raw_df.empty:
        print("Exiting script as no data was fetched.")
        return
        
    all_learners_data = process_learner_data(raw_df)
    if not all_learners_data:
        print("Exiting script as no learner data could be processed.")
        return

    # Calculate aggregate stats once
    agg_stats = calculate_aggregate_stats(all_learners_data)

    # Generate individual reports
    print("\nGenerating individual learner reports...")
    for data in all_learners_data.values():
        generate_individual_report(data, agg_stats, OUTPUT_DIR)
    
    # Generate master report
    print("\nGenerating master report...")
    first_learner = all_learners_data[list(all_learners_data.keys())[0]]
    generate_master_report(
        agg_stats, 
        first_learner['all_kcs'], 
        first_learner['all_metrics'], 
        len(all_learners_data), 
        OUTPUT_DIR
    )
    
    print(f"\n✅ All reports have been generated and saved in the '{OUTPUT_DIR}' folder.")

if __name__ == "__main__":
    main()

Starting report generation script...
Fetching data for learners starting with 'BT'...
Successfully fetched 107 records.
Processing raw data...
  -> Processing data for BT8BT9 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT15BT16 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT14 on Goal: 'Price with Purpose'
  -> Processing data for BT2 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT4 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT13 on Goal: 'Price with Purpose'
  -> Processing data for BT7 on Goal: 'Price with Purpose'
  -> Processing data for BT1 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT11BT12 on Goal: 'Price with Purpose'
  -> Processing data for BT7 on Goal: 'Price with Purpose'
  -> Processing data for BT5 on Goal: 'Price with Purpose'
  -> Processing data for BT3+BT6 on Goal: 'Price with Purpose'
  -> Processing data for BT10 on Goal: 'Break Even and Build Trust'
Data processing comple

In [6]:
# generate_reports.py

import os
import io
from pathlib import Path
from datetime import datetime, timedelta

# Data and DB Libraries
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

# Reporting and Plotting Libraries
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
OUTPUT_DIR = Path("GeneratedReports")
LEARNER_PREFIX = "BT"

# --- HELPER FUNCTIONS ---

def add_df_to_doc(doc, df, title=""):
    # (This function remains the same as the previous version)
    if title:
        doc.add_heading(title, level=2)
    if df.empty:
        doc.add_paragraph("No data available.")
        return
    df_rounded = df.copy()
    for col in df_rounded.select_dtypes(include=['number']).columns:
        df_rounded[col] = df_rounded[col].round(2)
    if isinstance(df_rounded.index, pd.RangeIndex):
        df_for_table = df_rounded.reset_index(drop=True)
    else:
        df_for_table = df_rounded.reset_index()
    table = doc.add_table(rows=1, cols=len(df_for_table.columns))
    table.style = 'Table Grid'
    for j, col_name in enumerate(df_for_table.columns):
        table.cell(0, j).text = str(col_name)
    for i, row in df_for_table.iterrows():
        row_cells = table.add_row().cells
        for j, val in enumerate(row):
            row_cells[j].text = str(val)

def create_plot_from_df(df, title, y_label, x_col='Decision Point'):
    # (This function remains the same as the previous version)
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 5.5))
    for col in df.columns:
        if col != x_col:
            ax.plot(df[x_col], df[col], marker='o', linestyle='-', label=col)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(x_col, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)
    if len(df.columns) > 2:
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.tight_layout(rect=[0, 0, 0.85, 1])
    else:
        fig.tight_layout()
    plt.xticks(rotation=45, ha='right')
    img_stream = io.BytesIO()
    plt.savefig(img_stream, format='png', dpi=300)
    plt.close(fig)
    img_stream.seek(0)
    return img_stream

# --- CORE SCRIPT LOGIC ---

def fetch_analytics_data(supabase: Client) -> pd.DataFrame:
    # (This function remains the same as the previous version)
    print(f"Fetching data for learners starting with '{LEARNER_PREFIX}'...")
    try:
        response = supabase.table('historical_learning_analytics').select(
            'created_at, user_id, goal_id, scenario_attempt_number, decision_number, '
            'kc_scores_after_decision, metric_values_after_decision, '
            'users!inner(name), goals!inner(name)'
        ).like('users.name', f'{LEARNER_PREFIX}%').order('user_id').order('created_at', desc=False).execute()
        if not response.data:
            print("No data found for the specified learners.")
            return pd.DataFrame()
        df = pd.DataFrame(response.data)
        df['user_name'] = df['users'].apply(lambda x: x['name'])
        df['goal_name'] = df['goals'].apply(lambda x: x['name'])
        df = df.drop(columns=['users', 'goals'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        print(f"Successfully fetched {len(df)} records.")
        return df
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return pd.DataFrame()

def process_learner_data(raw_df: pd.DataFrame) -> dict:
    # (This function remains the same as the previous version)
    print("Processing raw data...")
    if raw_df.empty: return {}
    all_learners_data = {}
    all_kcs = set()
    all_metrics = set()
    raw_df['kc_scores_after_decision'].dropna().apply(lambda x: all_kcs.update(x.keys()))
    raw_df['metric_values_after_decision'].dropna().apply(lambda x: all_metrics.update(x.keys()))
    all_kcs, all_metrics = sorted(list(all_kcs)), sorted(list(all_metrics))
    for (user_id, goal_id), group in raw_df.groupby(['user_id', 'goal_id']):
        user_name = group['user_name'].iloc[0]
        goal_name = group['goal_name'].iloc[0]
        print(f"  -> Processing data for {user_name} on Goal: '{goal_name}'")
        learner_df = group.sort_values('created_at').reset_index(drop=True)
        kc_df = pd.json_normalize(learner_df['kc_scores_after_decision']).reindex(columns=all_kcs).fillna(0)
        metric_df = pd.json_normalize(learner_df['metric_values_after_decision']).reindex(columns=all_metrics).fillna(0)
        learner_df['Decision Point'] = [f"G{r.goal_id}-S{r.scenario_attempt_number}-D{r.decision_number}" for r in learner_df.itertuples()]
        genesis_row = {'created_at': learner_df['created_at'].iloc[0] - timedelta(seconds=1), 'Decision Point': 'Genesis', **{kc: 0 for kc in all_kcs}, **{metric: 0 for metric in all_metrics}}
        genesis_df = pd.DataFrame([genesis_row])
        processed_df = pd.concat([learner_df[['created_at', 'Decision Point']], kc_df, metric_df], axis=1)
        full_df = pd.concat([genesis_df, processed_df], ignore_index=True)
        final_scores = full_df.iloc[-1]
        active_kcs = [kc for kc in all_kcs if final_scores[kc] != 0]
        active_metrics = [metric for metric in all_metrics if final_scores[metric] != 0]
        all_learners_data[f"{user_id}_{goal_id}"] = {"user_name": user_name, "goal_name": goal_name, "full_data": full_df, "all_kcs": all_kcs, "all_metrics": all_metrics, "active_kcs": active_kcs, "active_metrics": active_metrics}
    print("Data processing complete.")
    return all_learners_data

# --- MODIFIED FUNCTION ---
def calculate_all_stats(all_learners_data: dict):
    """Calculates statistics PER GOAL and returns a combined final scores DataFrame."""
    print("Calculating statistics per goal...")
    if not all_learners_data:
        return {}, pd.DataFrame()

    final_scores_list = []
    for data in all_learners_data.values():
        final_row = data['full_data'].iloc[-1].copy()
        final_row['user_name'] = data['user_name']
        final_row['goal_name'] = data['goal_name'] # Keep goal name for grouping
        final_scores_list.append(final_row)
    
    final_scores_df = pd.DataFrame(final_scores_list)
    all_kcs = all_learners_data[list(all_learners_data.keys())[0]]['all_kcs']
    all_metrics = all_learners_data[list(all_learners_data.keys())[0]]['all_metrics']

    per_goal_stats = {}
    for goal_name, group_df in final_scores_df.groupby('goal_name'):
        print(f"  -> Calculating stats for goal: '{goal_name}'")
        kc_stats = group_df[all_kcs].describe().loc[['mean', 'min', 'max']]
        metric_stats = group_df[all_metrics].describe().loc[['mean', 'min', 'max']]
        per_goal_stats[goal_name] = {'kc': kc_stats, 'metric': metric_stats}
        
    return per_goal_stats, final_scores_df

# --- MODIFIED FUNCTION ---
def generate_individual_report(learner_data: dict, per_goal_stats: dict, output_dir: Path):
    """Generates an individual report with goal-specific stats and split metric plots."""
    user_name = learner_data['user_name']
    goal_name = learner_data['goal_name']
    df = learner_data['full_data']
    active_kcs = learner_data['active_kcs']
    active_metrics = learner_data['active_metrics']

    print(f"  -> Generating report for {user_name} (Goal: {goal_name})...")
    
    doc = Document()
    doc.add_heading(f'Learner Report: {user_name}', 0)
    doc.add_paragraph(f"Goal Attempted: {goal_name}", style='Quote')
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    doc.add_heading('Final Cumulative Scores', level=1)
    doc.add_paragraph(
        "This section shows the learner's final score compared to the average, min, and max "
        "of all learners who attempted THIS SAME GOAL."
    )
    
    final_scores = df.iloc[-1]
    # Get stats for the specific goal this learner attempted
    specific_goal_stats = per_goal_stats.get(goal_name, {})

    if specific_goal_stats:
        # Final KC Scores Table
        if active_kcs:
            kc_summary = final_scores[active_kcs].to_frame(name='Your Score')
            kc_summary = kc_summary.join(specific_goal_stats['kc'][active_kcs].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
            add_df_to_doc(doc, kc_summary, "Final KC Scores")

        # Final Metric Scores Table
        if active_metrics:
            metric_summary = final_scores[active_metrics].to_frame(name='Your Value')
            metric_summary = metric_summary.join(specific_goal_stats['metric'][active_metrics].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
            add_df_to_doc(doc, metric_summary, "Final Metric Values")

    doc.add_page_break()
    doc.add_heading('Performance Over Time', level=1)
    
    # KC Scores Progression
    if active_kcs:
        kc_progression_df = df[['Decision Point'] + active_kcs]
        add_df_to_doc(doc, kc_progression_df.set_index('Decision Point'), "KC Scores per Decision Point")
        kc_plot_img = create_plot_from_df(kc_progression_df, "KC Scores Over Time", "KC Score")
        doc.add_picture(kc_plot_img, width=Inches(6.5))
    
    # --- NEW SPLIT METRIC PLOTTING LOGIC ---
    if active_metrics:
        metric_progression_df = df[['Decision Point'] + active_metrics]
        add_df_to_doc(doc, metric_progression_df.set_index('Decision Point'), "Metric Values per Decision Point")

        # Plot 1: Revenue
        if 'Revenue' in active_metrics:
            revenue_df = df[['Decision Point', 'Revenue']]
            revenue_plot_img = create_plot_from_df(revenue_df, "Revenue Over Time", "Revenue Value")
            doc.add_heading("Revenue Progression", level=3)
            doc.add_picture(revenue_plot_img, width=Inches(6.5))

        # Plot 2: Reputation
        if 'Reputation' in active_metrics:
            reputation_df = df[['Decision Point', 'Reputation']]
            rep_plot_img = create_plot_from_df(reputation_df, "Reputation Over Time", "Reputation (1-5 scale)")
            doc.add_heading("Reputation Progression", level=3)
            doc.add_picture(rep_plot_img, width=Inches(6.5))

        # Plot 3: Other high-scale metrics
        other_metrics = [m for m in ['Customer Satisfaction', 'Risk-Taking', 'Ethical Decision Making'] if m in active_metrics]
        if other_metrics:
            other_metrics_df = df[['Decision Point'] + other_metrics]
            other_plot_img = create_plot_from_df(other_metrics_df, "Other Metric Values Over Time", "Metric Value (0-100 scale)")
            doc.add_heading("High-Scale Metrics Progression", level=3)
            doc.add_picture(other_plot_img, width=Inches(6.5))

    file_path = output_dir / f"Report_{user_name.replace(' ', '_')}_{goal_name.replace(' ', '_')}.docx"
    doc.save(file_path)

# --- MODIFIED FUNCTION ---
def generate_master_report(final_scores_df: pd.DataFrame, all_kcs: list, all_metrics: list, num_learners: int, output_dir: Path):
    """Generates the master report with global stats and leaderboards."""
    print("-> Generating Master Report...")
    if final_scores_df.empty:
        print("   No data available to generate a master report.")
        return

    doc = Document()
    doc.add_heading('Master Learner Report', 0)
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    doc.add_paragraph(f"This report covers {num_learners} learner-goal attempts for names starting with '{LEARNER_PREFIX}'.")

    # Aggregate stats are now calculated here for the global view
    doc.add_heading('Global Aggregate Statistics (Final Scores)', level=1)
    kc_stats = final_scores_df[all_kcs].describe().loc[['mean', 'min', 'max']]
    metric_stats = final_scores_df[all_metrics].describe().loc[['mean', 'min', 'max']]
    add_df_to_doc(doc, kc_stats.T, "Global KC Score Statistics")
    add_df_to_doc(doc, metric_stats.T, "Global Metric Value Statistics")
    
    doc.add_page_break()
    
    doc.add_heading('Leaderboards', level=1)
    final_scores_df = final_scores_df.set_index('user_name')
    
    # KC Leaderboard
    final_scores_df['Total KC Score'] = final_scores_df[all_kcs].sum(axis=1)
    add_df_to_doc(doc, final_scores_df[['Total KC Score']].sort_values('Total KC Score', ascending=False).head(10), "Top 10 - KC Leaderboard")
    
    # Metric Leaderboard
    try:
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        normalized_metrics = scaler.fit_transform(final_scores_df[all_metrics])
        final_scores_df['Composite Metric Score'] = pd.DataFrame(normalized_metrics, index=final_scores_df.index).sum(axis=1)
        doc.add_paragraph("The Metric Leaderboard is calculated by normalizing each metric globally and summing these scores.")
        add_df_to_doc(doc, final_scores_df[['Composite Metric Score']].sort_values('Composite Metric Score', ascending=False).head(10), "Top 10 - Metric Leaderboard")
    except ImportError:
        doc.add_paragraph("Metric Leaderboard requires scikit-learn: pip install scikit-learn")

    doc.save(output_dir / "Master_Report.docx")
    print("   Master Report saved.")

# --- MODIFIED MAIN FUNCTION ---
def main():
    """Main function to run the entire reporting script."""
    print("Starting report generation script...")
    load_dotenv('.env.local')
    
    supabase_url = os.environ.get("NEXT_PUBLIC_SUPABASE_URL")
    supabase_key = os.environ.get("NEXT_PUBLIC_SUPABASE_ANON_KEY")

    if not supabase_url or not supabase_key:
        print("Error: Supabase credentials not found. Check .env.local file.")
        return

    OUTPUT_DIR.mkdir(exist_ok=True)
    supabase: Client = create_client(supabase_url, supabase_key)

    raw_df = fetch_analytics_data(supabase)
    if raw_df.empty: return
        
    all_learners_data = process_learner_data(raw_df)
    if not all_learners_data: return

    # Calculate stats (per goal and combined)
    per_goal_stats, final_scores_df = calculate_all_stats(all_learners_data)

    # Generate individual reports
    print("\nGenerating individual learner reports...")
    for data in all_learners_data.values():
        generate_individual_report(data, per_goal_stats, OUTPUT_DIR)
    
    # Generate master report
    print("\nGenerating master report...")
    first_learner = all_learners_data[list(all_learners_data.keys())[0]]
    generate_master_report(
        final_scores_df, 
        first_learner['all_kcs'], 
        first_learner['all_metrics'], 
        len(all_learners_data), 
        OUTPUT_DIR
    )
    
    print(f"\n✅ All reports have been generated and saved in the '{OUTPUT_DIR}' folder.")

if __name__ == "__main__":
    main()

Starting report generation script...
Fetching data for learners starting with 'BT'...
Successfully fetched 107 records.
Processing raw data...
  -> Processing data for BT8BT9 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT15BT16 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT14 on Goal: 'Price with Purpose'
  -> Processing data for BT2 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT4 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT13 on Goal: 'Price with Purpose'
  -> Processing data for BT7 on Goal: 'Price with Purpose'
  -> Processing data for BT1 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT11BT12 on Goal: 'Price with Purpose'
  -> Processing data for BT7 on Goal: 'Price with Purpose'
  -> Processing data for BT5 on Goal: 'Price with Purpose'
  -> Processing data for BT3+BT6 on Goal: 'Price with Purpose'
  -> Processing data for BT10 on Goal: 'Break Even and Build Trust'
Data processing comple

In [ ]:
# generate_reports.py

import os
import io
from pathlib import Path
from datetime import datetime, timedelta

# Data and DB Libraries
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

# Reporting and Plotting Libraries
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
OUTPUT_DIR = Path("GeneratedReports")
LEARNER_PREFIX = "BT"

# --- HELPER FUNCTIONS ---

def add_df_to_doc(doc, df, title=""):
    if title:
        doc.add_heading(title, level=2)
    if df.empty:
        doc.add_paragraph("No data available.")
        return
    df_rounded = df.copy()
    for col in df_rounded.select_dtypes(include=['number']).columns:
        df_rounded[col] = df_rounded[col].round(2)
    if isinstance(df_rounded.index, pd.RangeIndex):
        df_for_table = df_rounded.reset_index(drop=True)
    else:
        df_for_table = df_rounded.reset_index()
    table = doc.add_table(rows=1, cols=len(df_for_table.columns))
    table.style = 'Table Grid'
    for j, col_name in enumerate(df_for_table.columns):
        table.cell(0, j).text = str(col_name)
    for i, row in df_for_table.iterrows():
        row_cells = table.add_row().cells
        for j, val in enumerate(row):
            row_cells[j].text = str(val)

def create_plot_from_df(df, title, y_label, x_col='Decision Point'):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 5.5))
    for col in df.columns:
        if col != x_col:
            ax.plot(df[x_col], df[col], marker='o', linestyle='-', label=col)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(x_col, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)
    # Add a legend only if there is more than one data series to plot
    if len(df.columns) > 2:
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.tight_layout(rect=[0, 0, 0.85, 1])
    else:
        fig.tight_layout()
    plt.xticks(rotation=45, ha='right')
    img_stream = io.BytesIO()
    plt.savefig(img_stream, format='png', dpi=300)
    plt.close(fig)
    img_stream.seek(0)
    return img_stream

# --- CORE SCRIPT LOGIC ---

def fetch_analytics_data(supabase: Client) -> pd.DataFrame:
    print(f"Fetching data for learners starting with '{LEARNER_PREFIX}'...")
    try:
        response = supabase.table('historical_learning_analytics').select(
            'created_at, user_id, goal_id, scenario_attempt_number, decision_number, '
            'kc_scores_after_decision, metric_values_after_decision, '
            'users!inner(name), goals!inner(name)'
        ).like('users.name', f'{LEARNER_PREFIX}%').order('user_id').order('created_at', desc=False).execute()
        if not response.data:
            print("No data found for the specified learners.")
            return pd.DataFrame()
        df = pd.DataFrame(response.data)
        df['user_name'] = df['users'].apply(lambda x: x['name'])
        df['goal_name'] = df['goals'].apply(lambda x: x['name'])
        df = df.drop(columns=['users', 'goals'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        print(f"Successfully fetched {len(df)} records.")
        return df
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return pd.DataFrame()

def process_learner_data(raw_df: pd.DataFrame) -> dict:
    print("Processing raw data...")
    if raw_df.empty: return {}
    all_learners_data = {}
    all_kcs = set()
    all_metrics = set()
    raw_df['kc_scores_after_decision'].dropna().apply(lambda x: all_kcs.update(x.keys()))
    raw_df['metric_values_after_decision'].dropna().apply(lambda x: all_metrics.update(x.keys()))
    all_kcs, all_metrics = sorted(list(all_kcs)), sorted(list(all_metrics))
    for (user_id, goal_id), group in raw_df.groupby(['user_id', 'goal_id']):
        user_name = group['user_name'].iloc[0]
        goal_name = group['goal_name'].iloc[0]
        print(f"  -> Processing data for {user_name} on Goal: '{goal_name}'")
        learner_df = group.sort_values('created_at').reset_index(drop=True)
        kc_df = pd.json_normalize(learner_df['kc_scores_after_decision']).reindex(columns=all_kcs).fillna(0)
        metric_df = pd.json_normalize(learner_df['metric_values_after_decision']).reindex(columns=all_metrics).fillna(0)
        learner_df['Decision Point'] = [f"G{r.goal_id}-S{r.scenario_attempt_number}-D{r.decision_number}" for r in learner_df.itertuples()]
        genesis_row = {'created_at': learner_df['created_at'].iloc[0] - timedelta(seconds=1), 'Decision Point': 'Genesis', **{kc: 0 for kc in all_kcs}, **{metric: 0 for metric in all_metrics}}
        genesis_df = pd.DataFrame([genesis_row])
        processed_df = pd.concat([learner_df[['created_at', 'Decision Point']], kc_df, metric_df], axis=1)
        full_df = pd.concat([genesis_df, processed_df], ignore_index=True)
        final_scores = full_df.iloc[-1]
        active_kcs = [kc for kc in all_kcs if final_scores[kc] != 0]
        active_metrics = [metric for metric in all_metrics if final_scores[metric] != 0]
        all_learners_data[f"{user_id}_{goal_id}"] = {"user_name": user_name, "goal_name": goal_name, "full_data": full_df, "all_kcs": all_kcs, "all_metrics": all_metrics, "active_kcs": active_kcs, "active_metrics": active_metrics}
    print("Data processing complete.")
    return all_learners_data

def calculate_all_stats(all_learners_data: dict):
    print("Calculating statistics per goal...")
    if not all_learners_data:
        return {}, pd.DataFrame()
    final_scores_list = []
    for data in all_learners_data.values():
        final_row = data['full_data'].iloc[-1].copy()
        final_row['user_name'] = data['user_name']
        final_row['goal_name'] = data['goal_name']
        final_scores_list.append(final_row)
    final_scores_df = pd.DataFrame(final_scores_list)
    all_kcs = all_learners_data[list(all_learners_data.keys())[0]]['all_kcs']
    all_metrics = all_learners_data[list(all_learners_data.keys())[0]]['all_metrics']
    per_goal_stats = {}
    for goal_name, group_df in final_scores_df.groupby('goal_name'):
        print(f"  -> Calculating stats for goal: '{goal_name}'")
        kc_stats = group_df[all_kcs].describe().loc[['mean', 'min', 'max']]
        metric_stats = group_df[all_metrics].describe().loc[['mean', 'min', 'max']]
        per_goal_stats[goal_name] = {'kc': kc_stats, 'metric': metric_stats}
    return per_goal_stats, final_scores_df

def generate_individual_report(learner_data: dict, per_goal_stats: dict, output_dir: Path):
    """Generates an individual report with goal-specific stats and smart, split metric plots."""
    user_name = learner_data['user_name']
    goal_name = learner_data['goal_name']
    df = learner_data['full_data']
    active_kcs = learner_data['active_kcs']
    active_metrics = learner_data['active_metrics']

    print(f"  -> Generating report for {user_name} (Goal: {goal_name})...")
    
    doc = Document()
    doc.add_heading(f'Learner Report: {user_name}', 0)
    doc.add_paragraph(f"Goal Attempted: {goal_name}", style='Quote')
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    doc.add_heading('Final Cumulative Scores', level=1)
    doc.add_paragraph(
        "This section shows the learner's final score compared to the average, min, and max "
        "of all learners who attempted THIS SAME GOAL."
    )
    
    final_scores = df.iloc[-1]
    specific_goal_stats = per_goal_stats.get(goal_name, {})

    if specific_goal_stats:
        if active_kcs:
            kc_summary = final_scores[active_kcs].to_frame(name='Your Score')
            kc_summary = kc_summary.join(specific_goal_stats['kc'][active_kcs].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
            add_df_to_doc(doc, kc_summary, "Final KC Scores")
        if active_metrics:
            metric_summary = final_scores[active_metrics].to_frame(name='Your Value')
            metric_summary = metric_summary.join(specific_goal_stats['metric'][active_metrics].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
            add_df_to_doc(doc, metric_summary, "Final Metric Values")

    doc.add_page_break()
    doc.add_heading('Performance Over Time', level=1)
    
    if active_kcs:
        kc_progression_df = df[['Decision Point'] + active_kcs]
        add_df_to_doc(doc, kc_progression_df.set_index('Decision Point'), "KC Scores per Decision Point")
        kc_plot_img = create_plot_from_df(kc_progression_df, "KC Scores Over Time", "KC Score")
        doc.add_picture(kc_plot_img, width=Inches(6.5))
    
    if active_metrics:
        metric_progression_df = df[['Decision Point'] + active_metrics]
        add_df_to_doc(doc, metric_progression_df.set_index('Decision Point'), "Metric Values per Decision Point")

        if 'Revenue' in active_metrics:
            revenue_df = df[['Decision Point', 'Revenue']]
            revenue_plot_img = create_plot_from_df(revenue_df, "Revenue Over Time", "Revenue Value")
            doc.add_heading("Revenue Progression", level=3)
            doc.add_picture(revenue_plot_img, width=Inches(6.5))

        if 'Reputation' in active_metrics:
            reputation_df = df[['Decision Point', 'Reputation']]
            rep_plot_img = create_plot_from_df(reputation_df, "Reputation Over Time", "Reputation (1-5 scale)")
            doc.add_heading("Reputation Progression", level=3)
            doc.add_picture(rep_plot_img, width=Inches(6.5))

        # --- FINAL REFINED PLOTTING LOGIC ---
        other_metrics = [m for m in ['Customer Satisfaction', 'Risk-Taking', 'Ethical Decision Making'] if m in active_metrics]
        if other_metrics:
            other_metrics_df = df[['Decision Point'] + other_metrics]
            
            # If only one metric, be specific. Otherwise, be generic and use a legend.
            if len(other_metrics) == 1:
                metric_name = other_metrics[0]
                plot_title = f"{metric_name} Progression"
                y_label = f"{metric_name} (0-100 scale)"
            else:
                plot_title = "High-Scale Metrics Progression"
                y_label = "Metric Value (0-100 scale)"
            
            doc.add_heading(plot_title, level=3)
            other_plot_img = create_plot_from_df(other_metrics_df, plot_title, y_label)
            doc.add_picture(other_plot_img, width=Inches(6.5))

    file_path = output_dir / f"Report_{user_name.replace(' ', '_')}_{goal_name.replace(' ', '_')}.docx"
    doc.save(file_path)


def generate_master_report(final_scores_df: pd.DataFrame, all_kcs: list, all_metrics: list, num_learners: int, output_dir: Path):
    print("-> Generating Master Report...")
    if final_scores_df.empty:
        print("   No data available to generate a master report.")
        return
    doc = Document()
    doc.add_heading('Master Learner Report', 0)
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    doc.add_paragraph(f"This report covers {num_learners} learner-goal attempts for names starting with '{LEARNER_PREFIX}'.")
    doc.add_heading('Global Aggregate Statistics (Final Scores)', level=1)
    kc_stats = final_scores_df[all_kcs].describe().loc[['mean', 'min', 'max']]
    metric_stats = final_scores_df[all_metrics].describe().loc[['mean', 'min', 'max']]
    add_df_to_doc(doc, kc_stats.T, "Global KC Score Statistics")
    add_df_to_doc(doc, metric_stats.T, "Global Metric Value Statistics")
    doc.add_page_break()
    doc.add_heading('Leaderboards', level=1)
    final_scores_df = final_scores_df.set_index('user_name')
    final_scores_df['Total KC Score'] = final_scores_df[all_kcs].sum(axis=1)
    add_df_to_doc(doc, final_scores_df[['Total KC Score']].sort_values('Total KC Score', ascending=False).head(10), "Top 10 - KC Leaderboard")
    try:
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        normalized_metrics = scaler.fit_transform(final_scores_df[all_metrics])
        final_scores_df['Composite Metric Score'] = pd.DataFrame(normalized_metrics, index=final_scores_df.index).sum(axis=1)
        doc.add_paragraph("The Metric Leaderboard is calculated by normalizing each metric globally and summing these scores.")
        add_df_to_doc(doc, final_scores_df[['Composite Metric Score']].sort_values('Composite Metric Score', ascending=False).head(10), "Top 10 - Metric Leaderboard")
    except ImportError:
        doc.add_paragraph("Metric Leaderboard requires scikit-learn: pip install scikit-learn")
    doc.save(output_dir / "Master_Report.docx")
    print("   Master Report saved.")

def main():
    print("Starting report generation script...")
    load_dotenv('.env.local')
    supabase_url = os.environ.get("NEXT_PUBLIC_SUPABASE_URL")
    supabase_key = os.environ.get("NEXT_PUBLIC_SUPABASE_ANON_KEY")
    if not supabase_url or not supabase_key:
        print("Error: Supabase credentials not found. Check .env.local file.")
        return
    OUTPUT_DIR.mkdir(exist_ok=True)
    supabase: Client = create_client(supabase_url, supabase_key)
    raw_df = fetch_analytics_data(supabase)
    if raw_df.empty: return
    all_learners_data = process_learner_data(raw_df)
    if not all_learners_data: return
    per_goal_stats, final_scores_df = calculate_all_stats(all_learners_data)
    print("\nGenerating individual learner reports...")
    for data in all_learners_data.values():
        generate_individual_report(data, per_goal_stats, OUTPUT_DIR)
    print("\nGenerating master report...")
    first_learner = all_learners_data[list(all_learners_data.keys())[0]]
    generate_master_report(
        final_scores_df, 
        first_learner['all_kcs'], 
        first_learner['all_metrics'], 
        len(all_learners_data), 
        OUTPUT_DIR
    )
    print(f"\n✅ All reports have been generated and saved in the '{OUTPUT_DIR}' folder.")

if __name__ == "__main__":
    main()

In [8]:
# generate_reports.py

import os
import io
from pathlib import Path
from datetime import datetime, timedelta

# Data and DB Libraries
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

# Reporting and Plotting Libraries
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
OUTPUT_DIR = Path("GeneratedReports")
LEARNER_PREFIX = "BT"

# --- HELPER FUNCTIONS ---

def add_df_to_doc(doc, df, title=""):
    if title:
        doc.add_heading(title, level=2)
    if df.empty:
        doc.add_paragraph("No data available.")
        return
    df_rounded = df.copy()
    for col in df_rounded.select_dtypes(include=['number']).columns:
        df_rounded[col] = df_rounded[col].round(2)
    if isinstance(df_rounded.index, pd.RangeIndex):
        df_for_table = df_rounded.reset_index(drop=True)
    else:
        df_for_table = df_rounded.reset_index()
    table = doc.add_table(rows=1, cols=len(df_for_table.columns))
    table.style = 'Table Grid'
    for j, col_name in enumerate(df_for_table.columns):
        table.cell(0, j).text = str(col_name)
    for i, row in df_for_table.iterrows():
        row_cells = table.add_row().cells
        for j, val in enumerate(row):
            row_cells[j].text = str(val)

def create_plot_from_df(df, title, y_label, x_col='Decision Point'):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 5.5))
    for col in df.columns:
        if col != x_col:
            ax.plot(df[x_col], df[col], marker='o', linestyle='-', label=col)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(x_col, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)
    # Add a legend only if there is more than one data series to plot
    if len(df.columns) > 2:
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.tight_layout(rect=[0, 0, 0.85, 1])
    else:
        fig.tight_layout()
    plt.xticks(rotation=45, ha='right')
    img_stream = io.BytesIO()
    plt.savefig(img_stream, format='png', dpi=300)
    plt.close(fig)
    img_stream.seek(0)
    return img_stream

# --- CORE SCRIPT LOGIC ---

def fetch_analytics_data(supabase: Client) -> pd.DataFrame:
    print(f"Fetching data for learners starting with '{LEARNER_PREFIX}'...")
    try:
        response = supabase.table('historical_learning_analytics').select(
            'created_at, user_id, goal_id, scenario_attempt_number, decision_number, '
            'kc_scores_after_decision, metric_values_after_decision, '
            'users!inner(name), goals!inner(name)'
        ).like('users.name', f'{LEARNER_PREFIX}%').order('user_id').order('created_at', desc=False).execute()
        if not response.data:
            print("No data found for the specified learners.")
            return pd.DataFrame()
        df = pd.DataFrame(response.data)
        df['user_name'] = df['users'].apply(lambda x: x['name'])
        df['goal_name'] = df['goals'].apply(lambda x: x['name'])
        df = df.drop(columns=['users', 'goals'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        print(f"Successfully fetched {len(df)} records.")
        return df
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return pd.DataFrame()

def process_learner_data(raw_df: pd.DataFrame) -> dict:
    print("Processing raw data...")
    if raw_df.empty: return {}
    all_learners_data = {}
    all_kcs = set()
    all_metrics = set()
    raw_df['kc_scores_after_decision'].dropna().apply(lambda x: all_kcs.update(x.keys()))
    raw_df['metric_values_after_decision'].dropna().apply(lambda x: all_metrics.update(x.keys()))
    all_kcs, all_metrics = sorted(list(all_kcs)), sorted(list(all_metrics))
    for (user_id, goal_id), group in raw_df.groupby(['user_id', 'goal_id']):
        user_name = group['user_name'].iloc[0]
        goal_name = group['goal_name'].iloc[0]
        print(f"  -> Processing data for {user_name} on Goal: '{goal_name}'")
        learner_df = group.sort_values('created_at').reset_index(drop=True)
        kc_df = pd.json_normalize(learner_df['kc_scores_after_decision']).reindex(columns=all_kcs).fillna(0)
        metric_df = pd.json_normalize(learner_df['metric_values_after_decision']).reindex(columns=all_metrics).fillna(0)
        learner_df['Decision Point'] = [f"G{r.goal_id}-S{r.scenario_attempt_number}-D{r.decision_number}" for r in learner_df.itertuples()]
        genesis_row = {'created_at': learner_df['created_at'].iloc[0] - timedelta(seconds=1), 'Decision Point': 'Genesis', **{kc: 0 for kc in all_kcs}, **{metric: 0 for metric in all_metrics}}
        genesis_df = pd.DataFrame([genesis_row])
        processed_df = pd.concat([learner_df[['created_at', 'Decision Point']], kc_df, metric_df], axis=1)
        full_df = pd.concat([genesis_df, processed_df], ignore_index=True)
        final_scores = full_df.iloc[-1]
        active_kcs = [kc for kc in all_kcs if final_scores[kc] != 0]
        active_metrics = [metric for metric in all_metrics if final_scores[metric] != 0]
        all_learners_data[f"{user_id}_{goal_id}"] = {"user_name": user_name, "goal_name": goal_name, "full_data": full_df, "all_kcs": all_kcs, "all_metrics": all_metrics, "active_kcs": active_kcs, "active_metrics": active_metrics}
    print("Data processing complete.")
    return all_learners_data

def calculate_all_stats(all_learners_data: dict):
    print("Calculating statistics per goal...")
    if not all_learners_data:
        return {}, pd.DataFrame()
    final_scores_list = []
    for data in all_learners_data.values():
        final_row = data['full_data'].iloc[-1].copy()
        final_row['user_name'] = data['user_name']
        final_row['goal_name'] = data['goal_name']
        final_scores_list.append(final_row)
    final_scores_df = pd.DataFrame(final_scores_list)
    all_kcs = all_learners_data[list(all_learners_data.keys())[0]]['all_kcs']
    all_metrics = all_learners_data[list(all_learners_data.keys())[0]]['all_metrics']
    per_goal_stats = {}
    for goal_name, group_df in final_scores_df.groupby('goal_name'):
        print(f"  -> Calculating stats for goal: '{goal_name}'")
        kc_stats = group_df[all_kcs].describe().loc[['mean', 'min', 'max']]
        metric_stats = group_df[all_metrics].describe().loc[['mean', 'min', 'max']]
        per_goal_stats[goal_name] = {'kc': kc_stats, 'metric': metric_stats}
    return per_goal_stats, final_scores_df

def generate_individual_report(learner_data: dict, per_goal_stats: dict, output_dir: Path):
    """Generates an individual report with goal-specific stats and smart, split metric plots."""
    user_name = learner_data['user_name']
    goal_name = learner_data['goal_name']
    df = learner_data['full_data']
    active_kcs = learner_data['active_kcs']
    active_metrics = learner_data['active_metrics']

    print(f"  -> Generating report for {user_name} (Goal: {goal_name})...")
    
    doc = Document()
    doc.add_heading(f'Learner Report: {user_name}', 0)
    doc.add_paragraph(f"Goal Attempted: {goal_name}", style='Quote')
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    doc.add_heading('Final Cumulative Scores', level=1)
    doc.add_paragraph(
        "This section shows the learner's final score compared to the average, min, and max "
        "of all learners who attempted THIS SAME GOAL."
    )
    
    final_scores = df.iloc[-1]
    specific_goal_stats = per_goal_stats.get(goal_name, {})

    if specific_goal_stats:
        if active_kcs:
            kc_summary = final_scores[active_kcs].to_frame(name='Your Score')
            kc_summary = kc_summary.join(specific_goal_stats['kc'][active_kcs].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
            add_df_to_doc(doc, kc_summary, "Final KC Scores")
        if active_metrics:
            metric_summary = final_scores[active_metrics].to_frame(name='Your Value')
            metric_summary = metric_summary.join(specific_goal_stats['metric'][active_metrics].T.rename(columns={'mean': 'Class Avg', 'min': 'Class Min', 'max': 'Class Max'}))
            add_df_to_doc(doc, metric_summary, "Final Metric Values")

    doc.add_page_break()
    doc.add_heading('Performance Over Time', level=1)
    
    if active_kcs:
        kc_progression_df = df[['Decision Point'] + active_kcs]
        add_df_to_doc(doc, kc_progression_df.set_index('Decision Point'), "KC Scores per Decision Point")
        kc_plot_img = create_plot_from_df(kc_progression_df, "KC Scores Over Time", "KC Score")
        doc.add_picture(kc_plot_img, width=Inches(6.5))
    
    if active_metrics:
        metric_progression_df = df[['Decision Point'] + active_metrics]
        add_df_to_doc(doc, metric_progression_df.set_index('Decision Point'), "Metric Values per Decision Point")

        if 'Revenue' in active_metrics:
            revenue_df = df[['Decision Point', 'Revenue']]
            revenue_plot_img = create_plot_from_df(revenue_df, "Revenue Over Time", "Revenue Value")
            doc.add_heading("Revenue Progression", level=3)
            doc.add_picture(revenue_plot_img, width=Inches(6.5))

        if 'Reputation' in active_metrics:
            reputation_df = df[['Decision Point', 'Reputation']]
            rep_plot_img = create_plot_from_df(reputation_df, "Reputation Over Time", "Reputation (1-5 scale)")
            doc.add_heading("Reputation Progression", level=3)
            doc.add_picture(rep_plot_img, width=Inches(6.5))

        # --- FINAL REFINED PLOTTING LOGIC ---
        other_metrics = [m for m in ['Customer Satisfaction', 'Risk-Taking', 'Ethical Decision Making'] if m in active_metrics]
        if other_metrics:
            other_metrics_df = df[['Decision Point'] + other_metrics]
            
            # If only one metric, be specific. Otherwise, be generic and use a legend.
            if len(other_metrics) == 1:
                metric_name = other_metrics[0]
                plot_title = f"{metric_name} Progression"
                y_label = f"{metric_name} (0-100 scale)"
            else:
                plot_title = "High-Scale Metrics Progression"
                y_label = "Metric Value (0-100 scale)"
            
            doc.add_heading(plot_title, level=3)
            other_plot_img = create_plot_from_df(other_metrics_df, plot_title, y_label)
            doc.add_picture(other_plot_img, width=Inches(6.5))

    file_path = output_dir / f"Report_{user_name.replace(' ', '_')}_{goal_name.replace(' ', '_')}.docx"
    doc.save(file_path)


def generate_master_report(final_scores_df: pd.DataFrame, all_kcs: list, all_metrics: list, num_learners: int, output_dir: Path):
    print("-> Generating Master Report...")
    if final_scores_df.empty:
        print("   No data available to generate a master report.")
        return
    doc = Document()
    doc.add_heading('Master Learner Report', 0)
    doc.add_paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    doc.add_paragraph(f"This report covers {num_learners} learner-goal attempts for names starting with '{LEARNER_PREFIX}'.")
    doc.add_heading('Global Aggregate Statistics (Final Scores)', level=1)
    kc_stats = final_scores_df[all_kcs].describe().loc[['mean', 'min', 'max']]
    metric_stats = final_scores_df[all_metrics].describe().loc[['mean', 'min', 'max']]
    add_df_to_doc(doc, kc_stats.T, "Global KC Score Statistics")
    add_df_to_doc(doc, metric_stats.T, "Global Metric Value Statistics")
    doc.add_page_break()
    doc.add_heading('Leaderboards', level=1)
    final_scores_df = final_scores_df.set_index('user_name')
    final_scores_df['Total KC Score'] = final_scores_df[all_kcs].sum(axis=1)
    add_df_to_doc(doc, final_scores_df[['Total KC Score']].sort_values('Total KC Score', ascending=False).head(10), "Top 10 - KC Leaderboard")
    try:
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        normalized_metrics = scaler.fit_transform(final_scores_df[all_metrics])
        final_scores_df['Composite Metric Score'] = pd.DataFrame(normalized_metrics, index=final_scores_df.index).sum(axis=1)
        doc.add_paragraph("The Metric Leaderboard is calculated by normalizing each metric globally and summing these scores.")
        add_df_to_doc(doc, final_scores_df[['Composite Metric Score']].sort_values('Composite Metric Score', ascending=False).head(10), "Top 10 - Metric Leaderboard")
    except ImportError:
        doc.add_paragraph("Metric Leaderboard requires scikit-learn: pip install scikit-learn")
    doc.save(output_dir / "Master_Report.docx")
    print("   Master Report saved.")

def main():
    print("Starting report generation script...")
    load_dotenv('.env.local')
    supabase_url = os.environ.get("NEXT_PUBLIC_SUPABASE_URL")
    supabase_key = os.environ.get("NEXT_PUBLIC_SUPABASE_ANON_KEY")
    if not supabase_url or not supabase_key:
        print("Error: Supabase credentials not found. Check .env.local file.")
        return
    OUTPUT_DIR.mkdir(exist_ok=True)
    supabase: Client = create_client(supabase_url, supabase_key)
    raw_df = fetch_analytics_data(supabase)
    if raw_df.empty: return
    all_learners_data = process_learner_data(raw_df)
    if not all_learners_data: return
    per_goal_stats, final_scores_df = calculate_all_stats(all_learners_data)
    print("\nGenerating individual learner reports...")
    for data in all_learners_data.values():
        generate_individual_report(data, per_goal_stats, OUTPUT_DIR)
    print("\nGenerating master report...")
    first_learner = all_learners_data[list(all_learners_data.keys())[0]]
    generate_master_report(
        final_scores_df, 
        first_learner['all_kcs'], 
        first_learner['all_metrics'], 
        len(all_learners_data), 
        OUTPUT_DIR
    )
    print(f"\n✅ All reports have been generated and saved in the '{OUTPUT_DIR}' folder.")

if __name__ == "__main__":
    main()

Starting report generation script...
Fetching data for learners starting with 'BT'...
Successfully fetched 107 records.
Processing raw data...
  -> Processing data for BT8BT9 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT15BT16 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT14 on Goal: 'Price with Purpose'
  -> Processing data for BT2 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT4 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT13 on Goal: 'Price with Purpose'
  -> Processing data for BT7 on Goal: 'Price with Purpose'
  -> Processing data for BT1 on Goal: 'Break Even and Build Trust'
  -> Processing data for BT11BT12 on Goal: 'Price with Purpose'
  -> Processing data for BT7 on Goal: 'Price with Purpose'
  -> Processing data for BT5 on Goal: 'Price with Purpose'
  -> Processing data for BT3+BT6 on Goal: 'Price with Purpose'
  -> Processing data for BT10 on Goal: 'Break Even and Build Trust'
Data processing comple